<a href="https://colab.research.google.com/github/DevanWestley/AI_CourseDEIE/blob/main/Text_Generation_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Week 13: Generating text for NLP using SimpleRNN**

#### Compiled by Devan Westley
**Diclaimer:** <br/> This modul is part of hands on supplement for "Artificial Intelligence" course in The Department of Electrical and Information Engineering, Faculty of Engineering, Universitas Gadjah Mada, Indonesia. All codes are for solely educational purposes to explain lecture materials and provide deeper understanding on the presented materials.

### Table of interest:
1. Introduction.
2. Import data and preprocessing.
3. Model Building.
4. Model Training and Prediction.
5. Conclusion.

This exemple was simply taken in the book **Deep Learning with Keras**, *by Antonio Gulli and Sujit Pal, 2017.* Slight modification was performed to load data directly from URL and to explain the code in the text section.


## 1. Introduction.

RNNs have been used extensively by the **natural language processing (NLP)** community for various applications. One such application is building language models. A language model allows us to predict the probability of a word in a text given the previous words. Language models are important for various higher level tasks such as machine translation, spelling correction, and so on.

A side effect of the ability to predict the next word given previous words is a generative model that allows us to generate text by ***sampling from the output probabilities***. In language modeling, our input is typically a sequence of words and the output is a sequence of predicted words. The training data used is existing unlabeled text, where we set the label **y(t)** at time **t** to be the input **x(t+1)** at time **t+1**.

We will train a character based language model on the text of ***Alice in Wonderland*** to predict the next character given **10** previous characters. We have chosen to build a character-based model here because it has a smaller vocabulary and trains quicker. The idea is the same as using a **word-based language model**, except we use characters instead of words. We will then use the trained model to generate some text in the same style.

Let's get started.

In [ ]:
!pip install keras

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf

In [ ]:
from __future__ import print_function
from keras.layers import Dense, Activation
from keras.layers import SimpleRNN
from keras.models import Sequential


## 2. Import data and preprocessing.

We read our input text from the text of ***Alice in Wonderland*** on the Project Gutenberg website (https://www.gutenberg.org/cache/epub/11/pg11.txt). The file contains line breaks and non-ASCII characters, so we do some preliminary cleanup and write out the contents into a variable called text.

In [ ]:
path_to_file = tf.keras.utils.get_file(
    'pg11.txt',
    'https://www.gutenberg.org/cache/epub/11/pg11.txt',
)

fin = open(path_to_file, 'rb')

#fin = open("../input/alice_in_wonderland.txt", 'rb')
lines = []
for line in fin:
    line = line.strip().lower()
    line = line.decode("ascii", "ignore")
    if len(line) == 0:
        continue
    lines.append(line)
fin.close()
text = " ".join(lines)

174355/174355 [==============================] - 1s 4us/step


Since we are building a character-level RNN, our vocabulary is the set of characters that occur in the text. Since we will be dealing with the indexes to these characters rather than the characters themselves, the following code snippet creates the necessary lookup tables:

In [ ]:
# set of characters that occur in the text
chars = set([c for c in text])
# Total items in our vocabulary
nb_chars = len(chars)
# lookup tables to deal with indexes of characters rather than the characters themselves.
char2index = dict((c, i) for i, c in enumerate(chars))
index2char = dict((i, c) for i, c in enumerate(chars))
print("total characters in vocabulary: ", nb_chars)

total characters in vocabulary:  55


The next step is to create the input and label texts. We do this by stepping through the text by a number of characters given by the `STEP` variable and then extracting a span of text whose size is determined by the `SEQLEN` variable. The next character after the span is our label character.

In [ ]:
SEQLEN = 10
STEP = 1
input_chars = []
label_chars = []
for i in range(0, len(text) - SEQLEN, STEP):
    input_chars.append(text[i:i + SEQLEN])
    label_chars.append(text[i + SEQLEN])

Using the preceding code, the input and label texts for the text `it turned into a pig` would look like this: <br/>
it turned -> i </br>
t turned i -> n </br>
turned in -> t </br>
turned int -> o </br>
urned into ->  </br>
rned into -> a </br>
ned into a -> </br>
ed into a -> p </br>
d into a p -> i </br>
into a pi -> g </br>
</br>
Note: space is counted as one character

The next step is to vectorize these input and label texts. Each row of the input to the RNN
corresponds to one of the input texts shown previously. There are `SEQLEN` characters in this input, and since our vocabulary size is given by `nb_chars`, we represent each input character as a **one-hot encoded vector** of size (`nb_chars`). Thus each input row is a tensor of size (`SEQLEN` and `nb_chars`).

Our output label is a single character, so similar to the way we represent each character of our input, it is represented as a **one-hot vector of size** (`nb_chars`). Thus, the shape of each label is `nb_chars`.

In [ ]:
## create two empty matrices for input (X) and output / label (y)
## Size of matrix X is 10 x 55. There are 10 rows of SEQLEN characters and 55 columns
## Size of matrix Y is 1 x 55.
X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool_)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool_)

# # If there is any character in input_chars, set the value to 1
# So the matrix is sparse matrix, all value are set to zeros except value
# that represents character in input and output
for i, input_char in enumerate(input_chars):
    for j, ch in enumerate(input_char):
        X[i, j, char2index[ch]] = 1
    y[i, char2index[label_chars[i]]] = 1


In [ ]:
print(SEQLEN, nb_chars)

10 55


In [ ]:
print(input_chars)

['the projec', 'he project', 'e project ', ' project g', 'project gu', 'roject gut', 'oject gute', 'ject guten', 'ect gutenb', 'ct gutenbe', 't gutenber', ' gutenberg', 'gutenberg ', 'utenberg e', 'tenberg eb', 'enberg ebo', 'nberg eboo', 'berg ebook', 'erg ebook ', 'rg ebook o', 'g ebook of', ' ebook of ', 'ebook of a', 'book of al', 'ook of ali', 'ok of alic', 'k of alice', " of alice'", "of alice's", "f alice's ", " alice's a", "alice's ad", "lice's adv", "ice's adve", "ce's adven", "e's advent", "'s adventu", 's adventur', ' adventure', 'adventures', 'dventures ', 'ventures i', 'entures in', 'ntures in ', 'tures in w', 'ures in wo', 'res in won', 'es in wond', 's in wonde', ' in wonder', 'in wonderl', 'n wonderla', ' wonderlan', 'wonderland', 'onderland ', 'nderland t', 'derland th', 'erland thi', 'rland this', 'land this ', 'and this e', 'nd this eb', 'd this ebo', ' this eboo', 'this ebook', 'his ebook ', 'is ebook i', 's ebook is', ' ebook is ', 'ebook is f', 'book is fo', 'ook 

In [ ]:
print(label_chars)

['t', ' ', 'g', 'u', 't', 'e', 'n', 'b', 'e', 'r', 'g', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'o', 'f', ' ', 'a', 'l', 'i', 'c', 'e', "'", 's', ' ', 'a', 'd', 'v', 'e', 'n', 't', 'u', 'r', 'e', 's', ' ', 'i', 'n', ' ', 'w', 'o', 'n', 'd', 'e', 'r', 'l', 'a', 'n', 'd', ' ', 't', 'h', 'i', 's', ' ', 'e', 'b', 'o', 'o', 'k', ' ', 'i', 's', ' ', 'f', 'o', 'r', ' ', 't', 'h', 'e', ' ', 'u', 's', 'e', ' ', 'o', 'f', ' ', 'a', 'n', 'y', 'o', 'n', 'e', ' ', 'a', 'n', 'y', 'w', 'h', 'e', 'r', 'e', ' ', 'i', 'n', ' ', 't', 'h', 'e', ' ', 'u', 'n', 'i', 't', 'e', 'd', ' ', 's', 't', 'a', 't', 'e', 's', ' ', 'a', 'n', 'd', ' ', 'm', 'o', 's', 't', ' ', 'o', 't', 'h', 'e', 'r', ' ', 'p', 'a', 'r', 't', 's', ' ', 'o', 'f', ' ', 't', 'h', 'e', ' ', 'w', 'o', 'r', 'l', 'd', ' ', 'a', 't', ' ', 'n', 'o', ' ', 'c', 'o', 's', 't', ' ', 'a', 'n', 'd', ' ', 'w', 'i', 't', 'h', ' ', 'a', 'l', 'm', 'o', 's', 't', ' ', 'n', 'o', ' ', 'r', 'e', 's', 't', 'r', 'i', 'c', 't', 'i', 'o', 'n', 's', ' ', 'w', 'h', 'a',

In [ ]:
print(char2index)

{'?': 0, 'c': 1, ' ': 2, '[': 3, '8': 4, 'a': 5, '0': 6, ',': 7, '9': 8, 'g': 9, '7': 10, ';': 11, 't': 12, ')': 13, ']': 14, 'f': 15, '3': 16, 'x': 17, '2': 18, "'": 19, '_': 20, 'm': 21, '!': 22, '6': 23, 'w': 24, 'l': 25, 'j': 26, 'n': 27, 's': 28, ':': 29, '#': 30, '.': 31, 'h': 32, '5': 33, '(': 34, 'y': 35, '$': 36, '/': 37, '%': 38, 'p': 39, 'q': 40, 'z': 41, 'r': 42, 'e': 43, 'b': 44, '1': 45, 'i': 46, '*': 47, '4': 48, 'u': 49, 'o': 50, '-': 51, 'v': 52, 'd': 53, 'k': 54}


In [ ]:
print("Our input after one-hot encoding: ")
print("\n")
print(X)

Our input after one-hot encoding: 


[[[False False False ... False False False]
  [False False False ... False False False]
  [False False False ... False False False]
  ...
  [False False False ... False False False]
  [False False False ... False False False]
  [False  True False ... False False False]]

 [[False False False ... False False False]
  [False False False ... False False False]
  [False False  True ... False False False]
  ...
  [False False False ... False False False]
  [False  True False ... False False False]
  [False False False ... False False False]]

 [[False False False ... False False False]
  [False False  True ... False False False]
  [False False False ... False False False]
  ...
  [False  True False ... False False False]
  [False False False ... False False False]
  [False False  True ... False False False]]

 ...

 [[False False False ... False False False]
  [False False  True ... False False False]
  [False False False ... False False False]
  ...
  [

In [ ]:
print("Our output after one-hot encoding: ")
print("\n")
print(y)

Our output after one-hot encoding: 


[[False False False ... False False False]
 [False False  True ... False False False]
 [False False False ... False False False]
 ...
 [False False False ... False False  True]
 [False False False ... False False False]
 [False False False ... False False False]]


## 3. Model Building.

* We define the RNN's output dimension to have a size of **128**. This is a hyper-parameter that needs to be determined by experimentation. In general, if we choose too small a size, then the model does not have sufficient capacity for generating good text, and you will see long runs of repeating characters or runs of repeating word groups. On the other hand, if the value chosen is too large, the model has too many parameters and needs a lot more data to train effectively.
* We want to return a single character as output, not a sequence of characters, so `return_sequences=False`.
* In addition, we set `unroll=True` because it improves performance on the TensorFlow backend.
* The RNN is connected to a dense (fully connected) layer. The dense layer has (nb_char) units, which emits scores for each of the characters in the vocabulary. The activation on the dense layer is a softmax, which normalizes the scores to probabilities. The character with the highest probability is chosen as the prediction.
* We compile the model with the categorical cross-entropy loss function, a good loss function for categorical outputs, and the RMSprop optimizer.

Now let's review the code of model building.


In [ ]:
HIDDEN_SIZE = 128
BATCH_SIZE = 128
NUM_ITERATIONS = 25
NUM_EPOCHS_PER_ITERATION = 1
NUM_PREDS_PER_EPOCH = 100

model = Sequential()
model.add(SimpleRNN(HIDDEN_SIZE, return_sequences=False,
                    input_shape=(SEQLEN, nb_chars),
                    unroll=True))
model.add(Dense(nb_chars))
model.add(Activation("softmax"))

model.compile(loss="categorical_crossentropy", optimizer="rmsprop")

## 4. Model Training and Prediction
* Our training approach is a little different from what we have seen so far. So far our approach has been to train a model for a fixed number of epochs, then evaluate it against a portion of held-out test data. Since we don't have any labeled data here, we train the model for an epoch(`NUM_EPOCHS_PER_ITERATION=1`) then test it. We continue training like this for **25** (`NUM_ITERATIONS=25`) iterations, stopping once we see intelligible output. So effectively, we are training for NUM_ITERATIONS epochs and testing the model after each epoch.

* Our test consists of generating a character from the model given a random input, then dropping the first character from the input and appending the predicted character from our previous run, and generating another character from the model. We continue this 100 times (`NUM_PREDS_PER_EPOCH=100`) and generate and print the resulting string. The string gives us an indication of the quality of the model.

Let's review the code.

In [ ]:
for iteration in range(NUM_ITERATIONS):
    print("=" * 50)
    print("Iteration #: %d" % (iteration))
    model.fit(X, y, batch_size=BATCH_SIZE, epochs=NUM_EPOCHS_PER_ITERATION)
    test_idx = np.random.randint(len(input_chars))
    test_chars = input_chars[test_idx]
    print("\nGenerating from seed: %s" % (test_chars))
    print(test_chars, end="")
    for i in range(NUM_PREDS_PER_EPOCH):
        Xtest = np.zeros((1, SEQLEN, nb_chars))
        for i, ch in enumerate(test_chars):
            Xtest[0, i, char2index[ch]] = 1
        pred = model.predict(Xtest, verbose=0)[0]
        ypred = index2char[np.argmax(pred)]
        print(ypred, end="")
        # move forward with test_chars + ypred
        test_chars = test_chars[1:] + ypred
print()

Iteration #: 0
1243/1243 [==============================] - 6s 3ms/step - loss: 2.3474

Generating from seed:  sure to h
 sure to he the s in the tor the tor the tor the tor the tor the tor the tor the tor the tor the tor the tor t==================================================
Iteration #: 1
1243/1243 [==============================] - 4s 3ms/step - loss: 2.0459

Generating from seed: d i never 
d i never the wast on the sald the wast on the sald the wast on the sald the wast on the sald the wast on the ==================================================
Iteration #: 2
1243/1243 [==============================] - 4s 3ms/step - loss: 1.9396

Generating from seed: t to the t
t to the the tor and the could the gored the way to son the way to son the way to son the way to son the way t==================================================
Iteration #: 3
1243/1243 [==============================] - 4s 3ms/step - loss: 1.8592

Generating from seed: r your wal
r your wall to the groped and her

As you can see, by the end of the 25th epoch, it has learned to spell reasonably well, although it has trouble expressing coherent thoughts. The amazing thing about this model is that it is character-based and has no knowledge of words, yet it learns to spell words that look like they might have come from the original text.

## 5. Conclusion.

Generating the next character or next word of text is not the only thing you can do with this sort of model. This kind of model has been successfully used to make **stock predictions** (for more information refer to the article: ***Financial Market Time Series Prediction with Recurrent Neural Networks, by A. Bernal, S. Fok, and R. Pidaparthi, 2012***) and generate classical music (for more information refer to the article: ***DeepBach: A Steerable Model for Bach Chorales Generation, by G. Hadjeres and F. Pachet, arXiv:1612.01010, 2016***), to name a few interesting applications. **Andrej Karpathy** covers a few other fun examples, such as generating fake Wikipedia pages, algebraic geometry proofs, and Linux source code in his blog post at: ***The Unreasonable Effectiveness of Recurrent Neural Networks at http://karpathy.github.io/2015/05/21/rnn-effectiveness/.***

### References:
* **Deep Learning with Keras**, *by Antonio Gulli and Sujit Pal, 2017.*


**Thanks.**
